In [3]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error 
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

import numpy as np
import warnings 
import keras.backend as K
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor
import pandas as pd
import gensim

wv = gensim.models.word2vec.Word2Vec.load("updated_google_corpus")

TESTING_MODE = True

# load dataset
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
test_data = test_data.drop("index",axis=1)
data = pd.concat([train_data,test_data],sort=False)

In [4]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [5]:
if TESTING_MODE:
    d = pd.get_dummies(train_data[["country","region_2","province","taster_name","variety"]])
    d["points"]= train_data["points"]
    d["id"] = train_data['id']
#     col_dic = dict()
#     for column in ['country','province','region_2','taster_name','variety']:
#         col_dic[column] = list(set(list(train_data[column].values)))

#     num_data = pd.DataFrame() 
#     for column in col_dic.keys():
#         num_data[column] = train_data[column].apply(lambda column_item:col_dic[column].index(column_item))
#     # num_data["price"] = train_data["price"]
#     num_data["points"] = train_data["points"]
#     num_data["id"] = train_data["id"]
#     num_data.head()
else:
#     col_dic = dict()
#     for column in ['country','province','region_2','taster_name','variety']:
#         col_dic[column] = list(set(list(data[column].values)))

#     num_data = pd.DataFrame() 
#     for column in col_dic.keys():
#         num_data[column] = data[column].apply(lambda column_item:col_dic[column].index(column_item))
#     num_data["points"] = data["points"]
#     num_data["id"] = data["id"]
#     num_data.head()
    d = pd.get_dummies(data[["country","region_2","province","taster_name","variety"]])
    d["points"]= data["points"]
    d["id"] = data['id']

In [6]:
if TESTING_MODE:
    description_vecs = train_data["description"].apply(lambda words:sum([wv[word.lower()] for word in words.split()])*1/len(words.split()))
    for i in range(300):
        d["vec_dim_"+str(i)]=description_vecs.apply(lambda v:v[i])
else:
    description_vecs = data["description"].apply(lambda words:sum([wv[word.lower()] for word in words.split()])*1/len(words.split()))
    for i in range(300):
        d["vec_dim_"+str(i)] = description_vecs.apply(lambda v:v[i])
        
        

In [5]:
if TESTING_MODE:
    X_train, X_test, y_train, y_test = train_test_split(d, train_data["price"], test_size=0.3, random_state=1) # 70% training and 30% test
else:
    X_train,y_train,X_test, y_test = d[:175000], data["price"][:175000], d[175000:], data["price"][175000:]
    
X_train = X_train.drop(["id"],axis=1)

In [7]:
model = Sequential()
model.add(Dense(100, input_dim=X_train.shape[1]))
model.add(Activation("sigmoid"))
model.add(Dropout(.5))
model.add(Dense(256))
model.add(Activation("sigmoid"))
model.add(Dropout(.5))
model.add(Dense(32))
model.add(Activation("sigmoid"))
model.add(Dropout(.5))
model.add(Dense(1))
model.add(Activation("relu"))


In [8]:
model.compile(optimizer='rmsprop',
              loss=root_mean_squared_error,
              metrics=['accuracy'])


In [9]:
ids = X_test["id"]
X_test = X_test.drop(["id"],axis=1)

In [10]:
callbacks = [EarlyStopping(monitor='val_loss', patience=5)]

In [11]:
model.fit(X_train, y_train, epochs=100, batch_size=32,validation_data=(X_test, y_test),shuffle=True,callbacks=callbacks)

Train on 122500 samples, validate on 52500 samples
Epoch 1/100
122500/122500 [==============================] - 191s 2ms/step - loss: 30.5178 - accuracy: 0.0258 - val_loss: 26.0909 - val_accuracy: 0.0306
Epoch 2/100
122500/122500 [==============================] - 191s 2ms/step - loss: 25.9200 - accuracy: 0.0318 - val_loss: 26.1244 - val_accuracy: 0.0371
Epoch 3/100
122500/122500 [==============================] - 205s 2ms/step - loss: 25.2484 - accuracy: 0.0338 - val_loss: 26.9144 - val_accuracy: 0.0167
Epoch 4/100
122500/122500 [==============================] - 192s 2ms/step - loss: 24.9058 - accuracy: 0.0346 - val_loss: 23.8858 - val_accuracy: 0.0364
Epoch 5/100
122500/122500 [==============================] - 183s 1ms/step - loss: 24.4798 - accuracy: 0.0350 - val_loss: 25.3134 - val_accuracy: 0.0444
Epoch 6/100
122500/122500 [==============================] - 188s 2ms/step - loss: 24.3620 - accuracy: 0.0359 - val_loss: 23.4032 - val_accuracy: 0.0378
Epoch 7/100
122500/122500 [====

In [12]:
y_pred = model.predict(X_test)

In [ ]:
result = pd.DataFrame()
result['id'] = ids
result["price"] = y_pred
from math import ceil
result["price"]=result["price"].apply(ceil)

result.to_csv('out_nn.csv',index=False)

In [13]:
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

RMSE: 31.905956374100075


In [ ]:
y_pred[np.argmin(y_pred)]

In [ ]:
np.argmin(y_pred)

In [ ]:
y_pred[34363]

In [ ]:
import seaborn as sns
# sns.pairplot(num_data)

In [ ]:
num_data["price"]=train_data["price"]
num_data.corr()["price"]